In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from IPython.display import clear_output
from matplotlib.pyplot import imshow
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as k
from tqdm.auto import tqdm
import tensorflow_addons as tfa

## Reading data

In [2]:
train_df = pd.read_csv('./data/train.csv')
train_df.head()

,id,path,label
0,TRAIN_000,./train/TRAIN_000.mp4,3
1,TRAIN_001,./train/TRAIN_001.mp4,0
2,TRAIN_002,./train/TRAIN_002.mp4,1
3,TRAIN_003,./train/TRAIN_003.mp4,4
4,TRAIN_004,./train/TRAIN_004.mp4,4


In [3]:
len(train_df)

610

In [4]:
content = []
y = []

for i in tqdm(range(len(train_df))):
    file = train_df.loc[i, 'id']
    fname = f'{file}.mp4'
    label = train_df.loc[i, 'label']
    content.append(fname)
    y.append(label)

  0%|          | 0/610 [00:00<?, ?it/s]

In [5]:
content = np.array(content)
y = np.array(y)

In [6]:
content.shape, y.shape

((610,), (610,))

In [7]:
def read_frames(root_folder, arr, each_nth=10):
    videos=[]
    for j  in range(len(arr)):
        clear_output()
        print(np.round(100*j/len(arr),3))
            
        vcap=cv2.VideoCapture(root_folder+arr[j])
        success=True
  
        frames=[]
        cnt=0
        while success:
            try:
              success,image=vcap.read()
              cnt+=1
              if cnt%each_nth==0:
                image=resize(image,(128,128))
                # image = image / 255.
                frames.append(image)
            except Exception as e:
                print(e)
        videos.append(frames)
    
    return videos

In [8]:
def select_frames(frames_arr , n=10):
    videos=[]
    for i in range(len(frames_arr)):
        frames=[]
        for t in np.linspace(0, len(frames_arr[i])-1, num=n):
            frames.append(frames_arr[i][int(t)])
        videos.append(frames)
        
    videos = np.array(videos)
    print(videos.shape)
    return videos

In [9]:
X_frames = read_frames('./data/train/', content)
X_frames = select_frames(X_frames, 30)

99.836


[ WARN:0@24.572] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@24.572] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin609 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@24.572] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@24.572] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16

(610, 30, 128, 128, 3)


In [10]:
_, xte, _, yte = train_test_split(X_frames, y, shuffle=True, test_size=0.2, random_state=42)

In [11]:
ytr = to_categorical(y, 5)
yte = to_categorical(yte, 5)

In [12]:
X_frames.shape, xte.shape, ytr.shape, yte.shape

((610, 30, 128, 128, 3), (122, 30, 128, 128, 3), (610, 5), (122, 5))

## Building model

* previous version
* 차이점: train data 100% 다 활용, image 픽셀 정규화 적용. epochs 더 많이. 이에 따른 Earlystopping callback 추가

In [13]:
def list_to_stack(xs):
    xs = tf.stack(xs, axis = 1)
    s = tf.shape(xs)

    return xs

In [14]:
ish=(30, 128, 128, 3)
  
xs=[]


inp = Input(ish)

for slice_indx in range(0,10,1):
  x=Lambda(lambda x: x[:, slice_indx])(inp)
  x=BatchNormalization(momentum=0.8)(x)
  x=Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(x)
  x=BatchNormalization(momentum=0.8)(x)
  x=MaxPooling2D(pool_size=2)(x)
  
  x=Conv2D(filters=30, kernel_size=3, padding='same', activation='relu')(x)
  x=BatchNormalization(momentum=0.8)(x)
  x=MaxPooling2D(pool_size=2)(x)
  x=Conv2D(filters=30, kernel_size=3, padding='same', activation='relu')(x)
    
  xs.append(x)
  

t=Lambda(list_to_stack)(xs)
t=Conv3D(50,3,padding='same')(t)
# t=BatchNormalization(momentum=0.8)(t) # 왜 에러나지?
target_shape=(10,32*32*50)
t=Reshape(target_shape)(t)
t=GRU(25, return_sequences=True)(t)
t=GRU(50, return_sequences=False,dropout=0.5)(t)

t=Dense(100,'relu')(t)
out=Dense(5, activation='softmax')(t)

model = Model(inputs=inp, outputs=out)
opt = tf.keras.optimizers.SGD(lr=0.0087)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
model.summary()

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-01-28 15:18:56.952444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-28 15:18:56.953046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 128, 12  0           []                               
                                8, 3)]                                                            
                                                                                                  
 lambda (Lambda)                (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 lambda_2 (Lambda)              (None, 128, 128, 3)  0           ['input_1[0][0]']            

/Users/hansanghui/opt/anaconda3/envs/dacon1/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [15]:
history = model.fit(X_frames, ytr, epochs=11, batch_size=10, validation_data=(xte, yte), shuffle  = True, 
                                                                              callbacks=tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'))

Epoch 1/11


2023-01-28 15:19:05.220527: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-28 15:19:07.157634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:07.895264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:07.954163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:08.043274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:08.124278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - ETA: 0s - loss: 1.6062 - accuracy: 0.2525

2023-01-28 15:19:29.802413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:30.368544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:19:30.412792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - 26s 380ms/step - loss: 1.6062 - accuracy: 0.2525 - val_loss: 1.4786 - val_accuracy: 0.5000
Epoch 2/11
61/61 [==============================] - 22s 360ms/step - loss: 1.4589 - accuracy: 0.4295 - val_loss: 1.1999 - val_accuracy: 0.7049
Epoch 3/11
61/61 [==============================] - 21s 339ms/step - loss: 1.2551 - accuracy: 0.5656 - val_loss: 0.9944 - val_accuracy: 0.7951
Epoch 4/11
61/61 [==============================] - 21s 343ms/step - loss: 1.0401 - accuracy: 0.7082 - val_loss: 0.7616 - val_accuracy: 0.8770
Epoch 5/11
61/61 [==============================] - 20s 332ms/step - loss: 0.8279 - accuracy: 0.7902 - val_loss: 0.5625 - val_accuracy: 0.8689
Epoch 6/11
61/61 [==============================] - 20s 327ms/step - loss: 0.6045 - accuracy: 0.8754 - val_loss: 0.3075 - val_accuracy: 0.9836
Epoch 7/11
61/61 [==============================] - 21s 337ms/step - loss: 0.4308 - accuracy: 0.9230 - val_loss: 0.2078 - val_accuracy: 0.9836
Epoch 8/11

In [16]:
from sklearn.metrics import f1_score

preds = model.predict(X_frames)
preds = np.argmax(preds, axis=1)
f1 = f1_score(preds, train_df['label'], average='macro')
print(f'f1-score: {f1:.4f}')

2023-01-28 15:23:08.291504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/20 [>.............................] - ETA: 24s

2023-01-28 15:23:08.883460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-28 15:23:08.940156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 5s 197ms/step
f1-score: 1.0000


* inference modeling1

In [17]:
test = pd.read_csv('./data/test.csv')
test.head()

,id,path
0,TEST_000,./test/TEST_000.mp4
1,TEST_001,./test/TEST_001.mp4
2,TEST_002,./test/TEST_002.mp4
3,TEST_003,./test/TEST_003.mp4
4,TEST_004,./test/TEST_004.mp4


In [18]:
test_content = []

for i in tqdm(range(len(test))):
    file = test.loc[i, 'id']
    fname = f'{file}.mp4'
    test_content.append(fname)

test_content = np.array(test_content)
test_frames = read_frames('./data/test/', test_content)
test_frames = select_frames(test_frames, 30)

t_preds = model.predict(test_frames)
t_preds = np.argmax(t_preds, axis=1)

99.346


[ WARN:0@286.416] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@286.416] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin762 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@286.416] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@286.417] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suit

(153, 30, 128, 128, 3)
5/5 [==============================] - 2s 310ms/step


* modeling1 submission

In [19]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['label'] = t_preds
submit.head()

,id,label
0,TEST_000,0
1,TEST_001,3
2,TEST_002,0
3,TEST_003,2
4,TEST_004,2


In [20]:
submit.to_csv('./modeling2.csv', index=False)

* train data 전부 사용, earlystopping condition 추가, 이미지 픽셀 정규화 적용 : 0.4254363181
* train data 전부 사용, earlystopping condition 추가 : 0.627346441

-> 흠,,, 이유를 모르겠다.

## EfficientNetB0

In [23]:
# net = tf.keras.applications.EfficientNetB0(include_top = False)
# net.trainable = False

# model2 = tf.keras.Sequential([
#     tf.keras.layers.Rescaling(scale=255),
#     tf.keras.layers.TimeDistributed(net),
#     tf.keras.layers.Dense(5),
#     tf.keras.layers.GlobalAveragePooling3D()
# ])

# model2.compile(optimizer = 'adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [24]:
# model2.fit(X_frames, ytr, 
#           epochs = 10,
#           batch_size=10,
#           validation_data = (xte, yte),
#           callbacks = tf.keras.callbacks.EarlyStopping(patience = 2, monitor = 'val_loss'))

* 이 친구는 다음 기회에

## I3D